In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({
    'text.usetex': True,
    'font.size': 18
})


from skimage.transform import radon, rescale

# images of size NxN
N = 50

numAng = 70

n =N**2
theta = np.linspace(0.0, 180, numAng, endpoint=False)

def A(v):
    return radon(v.reshape(N,N),theta=theta,preserve_range=True)
    


In [ ]:




#v = np.random.randn(n)
v = np.ones(n)
v /= np.linalg.norm(v)
Av = A(v)
normAv = np.linalg.norm(Av,'fro')

n = np.prod(v.shape)
m = np.prod(Av.shape)

print(f'Radon transform, input dim {n}, output dim {m}')

# list of estimates
nA = []

# list of stepsizes
taus = []

nA.append(normAv)
maxiter = 10*n
check = 10

for i in range(maxiter):
    # sample x orthogonal to v with unit norm
    x = np.random.randn(n)
    x -= np.sum(x*v)*v
    x /= np.linalg.norm(x)
    Ax = A(x)
    normAx = np.linalg.norm(Ax,'fro')
    vOld = v.copy()
    a = np.sum(Ax*Av)
    b = normAx**2 - normAv**2
    tau = np.sign(a)*(b/(2*np.abs(a)) + np.sqrt(b**2/(4*a**2)+1))
    v += tau*x
    v /= np.linalg.norm(v)
    Av = A(v)
    normAv = np.linalg.norm(Av,'fro')
    nA.append(normAv)
    taus.append(tau)
    #aS.append(a)
    if i%check==0:
        print(f'Step {i: 5d}. Est: {nA[-1]:2.2f}, (step with size {tau:2.2e})')

vAdjointFree = v

In [ ]:
import numpy as np
from skimage.transform import radon, iradon

np.random.seed(4242)
theta = np.linspace(0.0, 180, numAng, endpoint=False)

v = np.random.randn(N,N)
w = np.random.randn(N,numAng)
Av = radon(v,theta=theta,preserve_range=True)

ATw = iradon(w, theta=theta, filter_name=None,preserve_range=True)


print(f'<v,A^Tw> = {np.sum(v*ATw)},  <Av,w> = {np.sum(Av*w)}')

In [ ]:
# do power method

def AT(w):
    return iradon(w,theta=theta,filter_name=None,preserve_range=True).reshape(n,1)


v = np.random.randn(n)
#v = np.ones(n)
v /= np.linalg.norm(n)
for k in range(20):
    Av = A(v)
    print(f'Step {k}. Est: {np.linalg.norm(Av):2.2f} Est: {np.sum(v*AT(Av)):2.2f} Est: {np.sqrt(np.linalg.norm(AT(Av))):2.2f}')
    ATAv = AT(Av)
    v = ATAv/np.linalg.norm(ATAv)

vPower = v

In [ ]:
plt.imshow(vAdjointFree.reshape(N,N),vmin=0, vmax=0.03)
plt.colorbar()
plt.show()
    
plt.imshow(vPower.reshape(N,N),vmin=0, vmax=0.03)
plt.colorbar()
plt.show()